# Data Exploration

In [1]:
import pandas as pd

Load Data:

In [2]:
complete2014Data = pd.DataFrame.from_csv('cache/complete2014Data.csv')

### 1. Get Basic Information About Dataset

Infos such as size of data, number of flights per months, number of delays in total and average.

*A flight delay is a when an airline flight takes off and/or lands later than its scheduled time. The Federal Aviation Administration (FAA) considers a flight to be delayed when it is 15 minutes later than its scheduled time.*

Source: https://en.wikipedia.org/wiki/Flight_cancellation_and_delay

Percentage of flights with arrival delays and departure delays:

In [15]:
departureDelayed = complete2014Data.DEP_DELAY >= 15
arrivalDelayed = complete2014Data.ARR_DELAY >= 15
bothDelayed = (complete2014Data.DEP_DELAY >= 15) & (complete2014Data.ARR_DELAY >= 15)

print "Percentage departure delays out of all flights: ", sum(departureDelayed)/float(len(complete2014Data))
print "Percentage arrival delays out of all flights: ", sum(arrivalDelayed)/float(len(complete2014Data))
print "Arrival delayed if departure was already delayed: ", sum(bothDelayed)/float(sum(arrivalDelayed))

Percentage departure delays out of all flights:  0.205397048117
Percentage arrival delays out of all flights:  0.213156062972
Arrival delayed if departure was already delayed:  0.78331081604


As we can see above, roughly 1/5 of all flights have a delayed departure, almost the same percentage of flights arrives late at the destination. Only a small percentage of delayed departures manage to catch up during the flight, since more than 78% of the delayed arrivals already suffered from a delayed departure.

###2. What Airports are the Worst/Best?

Airport ranking (origina/destination) regarding delay times.
Differentiate between different delay types (weather, security, etc.), e.g. are certain airports more affected by weather delays than others?

###3. What Airlines are the Worst/Best? What About Specific Routes?

Delay ranking regarding airlines and routes.

###4. How Does Weather Affect Delays?

Combine with external weather data and show interesting correlations